In [2]:
import autokeras as ak # We need this otherwise model doesn't load :(
from keras.models import load_model
from PIL import Image
from numpy import expand_dims, exp
import tqdm
import os
import pandas as pd
price_model = load_model("model-full128_trainlong.h5", compile=False)

IMG_INPUT_SIZE = (128, 128)
def score(img):
    prediction = price_model.predict(expand_dims(img.resize(IMG_INPUT_SIZE), axis=0))
    # Values are log normalized
    return exp(prediction).item(0)

2023-02-23 12:14:39.698226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 12:14:42.074750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 12:14:42.076264: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
if "output_prices.csv" in os.listdir():
    output_prices_df = pd.read_csv("output_prices.csv")
else:
    output_prices_df = pd.DataFrame(columns=["price"])
indices_done = list(output_prices_df.index.values.astype(int))

for file in [filename for filename in os.listdir("Images") if filename[-1] == 'g']:
    index = int(file[:-4])
    
    if index not in indices_done:
        with Image.open("Images/"+file) as im:
            try:
                price = score(im)
            except:
                price = None
            output_prices_df.loc[index] = price
        output_prices_df.to_csv("output_prices.csv", index=None)

In [6]:
artworks_df = pd.read_csv("artworks_with_prices_v3.csv")
generated_df = pd.read_csv("Generated.csv")
generated_scores_df = pd.read_csv("output_prices.csv")

# add artists to generated scores
generated_scores_df["artist"] = None
generated_scores_df["source_artwork"] = None
indices = range(len(generated_scores_df))
for index in indices:
    source_artwork_index = generated_df.source_artwork.iloc[index]
    generated_scores_df.source_artwork[index] = str(artworks_df.name.iloc[source_artwork_index])
    generated_scores_df.artist[index]  = str(artworks_df.artist_name.iloc[source_artwork_index])

generated_scores_df.to_csv("output_prices.csv", index=None)

/tmp/ipykernel_55825/71235130.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_scores_df.source_artwork[index] = str(artworks_df.name.iloc[source_artwork_index])
/tmp/ipykernel_55825/71235130.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_scores_df.artist[index]  = str(artworks_df.artist_name.iloc[source_artwork_index])
